In [1]:
import os, re, sys
import pymongo
import collections
from com.ibm.watson import DESKTOP, DOCS_DIR
import com.ibm.watson.utils.pubmed_utils as pu

In [8]:
base_dir = os.path.join(DOCS_DIR, "Curation", "pubmed")
pmids = []
pmid_regex = re.compile("\d{5,}")
with open(os.path.join(base_dir, "temp.txt")) as f:
    for line in f:
        pmids.extend(pmid_regex.findall(line.strip()))
    
pmids = set(pmids)
print len(pmids)
with open(os.path.join(base_dir, "pmids.txt"), "wb") as f:
    [f.write(pmid.strip() + "\n") for pmid in pmids]

812


In [9]:
medline_articles = pu.get_medline(list(pmids))
    
output_f = os.path.join(base_dir, "pubmed_mesh_terms.tsv")
substance_regex = re.compile("(\()(.*)(\))")
with open(output_f, "wb") as f:
    for article in medline_articles:
        pmid = article.get("PMID")
        substances = [ match.group(2) for match in [substance_regex.search(substance) for substance in article.get("RN", [])] if match ]
        to_write = {"Mesh Term" : article.get("MH", []), "Publication Types" : article.get("PT", []), "Substances" : substances}
        for (key, values) in to_write.items():
            for value in values:
                #print "\t".join([ pmid, key, value, "\n" ])
                f.write("\t".join([ pmid, key, value, "\n" ]))